In [136]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider
import matplotlib.pyplot as plt
import ipywidgets as widgets

mean1 = np.array([-5., 0])
mean2 = np.array([5., 0])
random_seed=42
np.random.seed(random_seed)


def sample_mixture(n_samples=1000, angle_deg=45, weight=0.5):
    
    
    angle_rad = np.deg2rad(angle_deg)
    
    
    cov1 = np.array([[2.0, 0.0],
                     [0.0, .25]])
    
    
    cov2_base = np.array([[2.0, 0.0],
                          [0.0, .25]])
    
    R = np.array([[np.cos(angle_rad), -np.sin(angle_rad)],
                  [np.sin(angle_rad),  np.cos(angle_rad)]])
    
    cov2 = R @ cov2_base @ R.T
    
    n1 = int(n_samples * weight)
    n2 = n_samples - n1
    
    samples1 = np.random.multivariate_normal(mean1, cov1, size=n1)
    samples2 = np.random.multivariate_normal(mean2, cov2, size=n2)
    
    # samples = np.vstack([samples1, samples2])
    # np.random.shuffle(samples)
    
    return samples1, samples2


def compute_new_W(X_s, X_p, y_s, y_p, lam=1.):
    A = X_s.T@X_s
    B = X_p.T@X_p
    X = np.vstack((X_s, X_p))
    y = np.hstack((y_s, y_p))
    # y = y.reshape(-1, 1)
    # y_s = y_s.reshape(-1, 1)
    # y_p = y_p.reshape(-1, 1)
    C = lam * (A@B + B@A) + X.T@X 
    # C0 = X.T@X 
    # z0 = X.T@y
   
    z = lam * (A@(X_p.T@y_p) + B@(X_s.T@y_s)) + X.T@y
    W_new = np.linalg.solve(C, z)
    
    # print(X.shape, y.shape, y_s.shape)

    # print(C, C0)
    # print(z, z0)
    # print(W_new, np.linalg.solve(C0, z0))
    return W_new


def angle_between(v1, v2, degrees=True):
    v1 = np.array(v1)
    v2 = np.array(v2)
    
    # Normalize dot product
    dot = np.dot(v1, v2)
    norm_prod = np.linalg.norm(v1) * np.linalg.norm(v2)
    cos_theta = np.clip(dot / norm_prod, -1.0, 1.0)  # Clip to handle numerical issues

    angle_rad = np.arccos(cos_theta)
    return np.degrees(angle_rad) if degrees else angle_rad


def plot_mixture(n_samples, angle_deg, split, W_r, W_theta, lam, sigma):
    X_s, X_p = sample_mixture(n_samples, angle_deg, split)
    plt.figure(figsize=(6, 6))
    plt.scatter(X_s[:, 0], X_s[:, 1], s=5, alpha=0.6, c="blue")
    plt.text(mean1[0], mean1[1]+2, r'$X_s$', fontsize=12, ha='left', va='bottom', color='black')
    plt.scatter(X_p[:, 0], X_p[:, 1], s=5, alpha=0.6, c="blue")
    plt.text(mean2[0], mean2[1]+2, r'$X_p$', fontsize=12, ha='left', va='bottom', color='black')
    centers = np.vstack((mean1, mean2))
    plt.scatter(centers[:, 0], centers[:, 1], s=10, c='red')
    plt.axis('equal')
    plt.title(f"Mixture of 2D Gaussians\nRotated Component at {angle_deg:.1f}°")
    plt.grid(True)
    plt.xlim(-11, 11)
    plt.ylim(-11, 11)
    W_theta = np.deg2rad(W_theta)
    W = np.array([W_r*np.cos(W_theta), W_r*np.sin(W_theta)])
    y_s, y_p = X_s@W + sigma * np.random.normal(len(X_s)), X_p@W + sigma * np.random.normal(len(X_p))

    plt.scatter(-10.*np.ones_like(y_s), y_s, s=8, alpha=0.6, c="darkgreen", marker="^")
    plt.text(-10.25, -7.5, r'$y_s$', fontsize=12, ha='left', va='bottom', color='black')
    plt.scatter(10.*np.ones_like(y_p), y_p, s=8, alpha=0.6, c="darkgreen", marker="^")
    plt.text(9.75, -7.5, r'$y_p$', fontsize=12, ha='left', va='bottom', color='black')
    
    plt.quiver(0, 0, W[0], W[1], angles='xy', scale_units='xy', scale=1, color='black')
    plt.text(W[0], W[1], r'$W$', fontsize=12, ha='left', va='bottom', color='black')


    W_new = compute_new_W(X_s, X_p, y_s, y_p, lam)
    print(r"lambda" + f"={lam}")
    print(rf'W={W}')
    print(rf'W_{{new}}={W_new}')
    print(f'Angle between W and W_new = {angle_between(W, W_new):.2f} degrees')
    plt.quiver(0, 0, W_new[0], W_new[1], angles='xy', scale_units='xy', scale=1, color='cyan', alpha=0.5)
    plt.text(W_new[0], W_new[1]+1, r'$W_{new}$', fontsize=12, ha='left', va='bottom', color='black')
    
    plt.show()



# Create the interactive widget
interact(plot_mixture,
        n_samples=IntSlider(min=500, max=10000, step=100, value=1000, description="Total samples",\
                        style={'description_width': 'auto'}, layout=widgets.Layout(width='50%')),\
        angle_deg=FloatSlider(min=0, max=180, step=1, value=45, description="Orientation of right mixture component (degree)",
                                 style={'description_width': 'auto'}, layout=widgets.Layout(width='50%')),\
        split=FloatSlider(min=0.1, max=1., step=0.1, value=0.5, description="Sample split",\
                        style={'description_width': 'auto'}, layout=widgets.Layout(width='50%')),\
        W_r=FloatSlider(min=0.1, max=5., step=0.1, value=45, description="Length of W",\
                        style={'description_width': 'auto'}, layout=widgets.Layout(width='50%')),
        W_theta=FloatSlider(min=0, max=180, step=1, value=45, description="Orientation of W",
                            style={'description_width': 'auto'}, layout=widgets.Layout(width='50%')),\
        lam=FloatSlider(min=0., max=1e-5, step=1e-6, value=1e-6, description="lambda",\
                        style={'description_width': 'auto'}, layout=widgets.Layout(width='50%')),\
        sigma=FloatSlider(min=0., max=1., step=0.01, value=0.01, description="noise strength",\
                        style={'description_width': 'auto'}, layout=widgets.Layout(width='50%')));

interactive(children=(IntSlider(value=1000, description='Total samples', layout=Layout(width='50%'), max=10000…